In [271]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

from bs4 import BeautifulSoup
from utils import *

path_2004 = "./2004/"
all_2004 = os.listdir(path_2004)
all_2004 = [x for x in all_2004 if ('DS_Store' not in x)]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [272]:
results = []
for fname in all_2004:
    test_f = open(path_2004 + fname)
    wholetext = test_f.read()
    cikbase = re.search(r"CENTRAL INDEX KEY:\s*?\d{10}\b", wholetext, re.I).group()
    cik_re = re.split(r'\s', cikbase)[-1]        

    datadatebase = re.search(r"CONFORMED PERIOD OF REPORT:\s*?\d{8}\b", wholetext, re.I).group() 
    datadate_re = re.split(r'\s', datadatebase)[-1]  

    wholetext = wholetext.lower()
    tbl_flag = 0 # 1 if "contractual obligations" table exists
    row_flag = 0 # 1 if "purchasing obligations" row exists
    table = scrape_table(wholetext)
    if table:
        tbl_flag = 1
        rows = scrape_rows(table)
    else:
        lines, tbl_list = scrape_text_table(wholetext)
        if tbl_list:
            tbl_flag = 1
            rows = scrape_text_rows(lines, tbl_list)
            
    for row in rows:
        row_flag = 1
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': row['total'], '<1': row['<1'], '1-3':row['1-3'], '3-5':row['3-5'], '>5':row['>5'], 
                  'category': row['category']}
        results.append(result)

    # No table or no row
    if (tbl_flag == 0) or (row_flag == 0):
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': '-', '<1': '-', '1-3':'-', '3-5':'-', '>5':'-'}

        results.append(result)
pd.DataFrame(results).to_csv('scraped_files_df')

Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Unknown error!
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Unknown error!
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Unknown error!
Unknown error!
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
Found a table
No HTML tables!
Found a table
No HTML tables!
Found a table
Found a table
Found a table
Found a table
Found a table


In [273]:
results_df = pd.DataFrame(results)
row_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 1)] # table but no row
none_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 0)] # no table no row
last_check = results_df[(results_df['row'] == 1) & (results_df['tbl'] == 1)] # table and row, other issue?
print(len(set(last_check['fname'])), len(set(row_check['fname'])), len(set(none_check['fname'])))

47 9 0


In [279]:
already_checked = ['0001063197', '0000075362', '0000754009', '0000740260', '0001099668', '0000832428', '0000855581' ]
last_check

,1-3,3-5,<1,>5,category,cik,datadate,fname,row,tbl,total
1,27,0,12,0,capital lease\n obligations,0000881464,20031231,0001104659-04-007271.txt,1,1,39
2,9220,10935,3073,31413,operating\n leases,0000881464,20031231,0001104659-04-007271.txt,1,1,54641
3,321,463,513,1969,operating leases,0000018072,20040930,0001047469-04-035525.txt,1,1,672
4,270890,95387,113500,619155,gas supply,0000018072,20040930,0001047469-04-035525.txt,1,1,139378
5,55728,294147,28571,407017,transportation,0000018072,20040930,0001047469-04-035525.txt,1,1,28571
6,1723,1431,963,9480,operating leases(2),0001173284,20031231,0001193125-04-051895.txt,1,1,13597
7,20141,12090,12380,14710,noncancellable operating leases,0000790051,20031231,0001104659-04-006998.txt,1,1,59321
8,0,0,12400,0,purchase obligations,0000790051,20031231,0001104659-04-006998.txt,1,1,12400
9,2183,911,1224,1808,operating leases \n(1),0000007332,20031231,0000007332-04-000020.txt,1,1,6126
10,4.774e+06,3.868e+06,2.752e+06,21321000,operating lease obligations,0001070772,20040630,0001104659-04-029370.txt,1,1,32715000


# Examining one file

In [276]:
transposed_files = ["0001104659-04-008879.txt", "0001104659-04-030276.txt"]
test_f = open(path_2004 + transposed_files[1])
wholetext = test_f.read()
wholetext = wholetext.lower()
table = scrape_table(wholetext)


Found a table


In [277]:
table_to_list(table)

[['payments due by period'],
 ['contractual\n  obligations (1)',
  'total',
  '2004',
  '2005 \x96 2006',
  '2007 \x96 2008',
  '2009 and\n  thereafter'],
 ['long-term obligations (2)', '270,000', '0', '0', '270,000', '0'],
 ['total', '270,000', '0', '0', '270,000', '0']]

In [270]:
column_name_elim_words = ['amounts due by fiscal year']
elim_subwords = ['\xa0','thousands']
table_headers = [x.text for x in tbl.find_all('th')]
table_headers = [x for x in table_headers if (x not in column_name_elim_words)]
final_table_headers = []
for header in table_headers:
    if not any(elim_word in header for elim_word in elim_subwords):
        final_table_headers.append(header)
final_table_headers

AttributeError: 'NoneType' object has no attribute 'find_all'

In [101]:
[x.text for x in tbl.find('tr').find_all('td')]

['\nin\n  thousands\n',
 '\n\xa0\n',
 '\ntotal\n',
 '\n\xa0\n',
 '\n2004\n',
 '\n\xa0\n',
 '\n2005\n',
 '\n\xa0\n',
 '\n2006\n',
 '\n\xa0\n',
 '\n2007\n',
 '\n\xa0\n',
 '\n2008\n',
 '\n\xa0\n',
 '\nthereafter\n',
 '\n\xa0\n']